# This script demonstrates how to train a M²VAE with a split MNIST data set and evaluate the Hyperparameter

In [47]:
import vae_tools.sanity
import vae_tools.viz
import vae_tools.callbacks
import vae_tools.loader
from vae_tools.mmvae import MmVae, ReconstructionLoss
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
vae_tools.sanity.check()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.datasets import mnist
import numpy as np
from scipy.stats import norm
# Set the seed for reproducible results
import vae_tools.sampling
vae_tools.sampling.set_seed(0)
# resize the notebook if desired
#vae_tools.nb_tools.notebook_resize()
import matplotlib
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
import glob
import pickle
import astetik

python version:  3.5.2
keras version: 2.2.4-tf
tensorflow version: 2.0.2
matplotlib uses:  module://ipykernel.pylab.backend_inline
No GPUs available


In [48]:
def get_evaluation(dump_loc = '/mnt/ssd_pcie/mmvae_mnist_split/0/'):
    #dump_loc = '/mnt/ssd_pcie/mmvae_mnist_split/Backup_of_wrong_mutal_beta/1/'

    df = pd.read_hdf(dump_loc + 'history.h5', key='df', mode='r+')
    #df


    #%%

    # Show the first example
    # idx = 1
    # hist = tf.keras.callbacks.History()
    # hist.history = df[[c for c in df.columns if c[:6]=='h_list']].to_dict('records')[idx]
    # vae_tools.viz.plot_losses(hist, dpi=96, figsize_height_factor = .325)
    # plt.show()

    ## Load the JSD and average it over all samples

    #%%

    prefix = 'jsd_'
    jsd = None
    for idx in df['index']:
        loc = dump_loc + 'jsd_' + str(idx) + '_ab.p'
        data = pickle.load( open( loc, "rb" ) )
        if jsd is None:
            jsd = {}
            for k in data.keys():
                jsd[prefix + k] = []
        for k in data.keys():
            #print(np.mean(js[k]))
            jsd[prefix + k].append(np.mean(data[k]))
    # Add the JSD to the pandas data frame
    df_jsd = pd.DataFrame(jsd, columns=jsd.keys())

    #%% md

    ## Load the GNB classifications and calculate the precision

    #%%

    #_df = df

    gnb = None
    prefix = 'gnb_prec_'
    _,_,_, y_test = vae_tools.loader.mnist_split()
    denom = len(y_test)
    for idx in df['index']:
        loc = dump_loc + 'bayes_classifier_' + str(idx) + '.p'
        data = pickle.load( open( loc, "rb" ) )
        if gnb is None:
            gnb = {}
            for k in data.keys():
                gnb[prefix + k] = []
        for k in data.keys():
            #print(np.mean(js[k]))
            gnb[prefix + k].append(np.sum((data[k] == y_test))/denom)
    # Add the JSD to the pandas data frame
    df_gnb = pd.DataFrame(gnb, columns=gnb.keys())


    # concat
    return (df, pd.concat([df, df_jsd, df_gnb], axis =1))



dfs = []
for idx in range(5):
    dfs.append(get_evaluation(dump_loc = '/mnt/ssd_pcie/jmvae_mnist_split/' + str(idx) + '/'))

# concat
_df = pd.concat([d[1] for d in dfs], axis =0)
df = pd.concat([d[0] for d in dfs], axis =0)
#(df, _df) = dfs[0]

# make beta-mutual linear
#_df['beta_mutual'] = _df['beta_mutual'].apply(lambda x: int(np.log10(x)))
#_df = _df[_df['beta_mutual'] != int(1)]

# correlation w/o validation
_df = _df.drop(["activation","reconstruction_loss_metrics","optimizer","index"], axis=1)
_df = _df[[c for c in _df.columns if c[:5] != 'h_val']]


astetik.corr(_df, color_grades=5, style='astetik', annot=True)
#_df

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/ssd_pcie/jmvae_mnist_split/0/jsd_0_ab.p'

In [ ]:

#df

#_df1 = _df[_df['beta_mutual'] == int(1)]
#_df1[['h_loss', 'jsd_test_ab_vs_b']]

In [ ]:
#_df_1 = _df[_df['beta_mutual'] == int(-1)]
#_df_1[['h_loss', 'jsd_test_ab_vs_b']]


In [ ]:
def plot_loss_surface(loss_str = '', hp_str = [''], df = None, axs = None, ngridx = 100, ngridy = 200, vmin=0., vmax=1., norm=None):
    import itertools
    import matplotlib.tri as tri
    #  Plot surfaces in one row
    df_loss = df[[loss_str] + hp_str]
    #df_loss = _df[['h_loss', 'beta_mutual']]
    #_df[_df['beta_mutual'] == int(1)]

    hp_str_pairs = list(itertools.combinations(hp_str, 2))

    print(hp_str_pairs)

    cs = []

    _vmin, _vmax = df_loss[loss_str].values.min(), df_loss[loss_str].values.max()
    for pair, ax in zip(hp_str_pairs, axs):

        z = df_loss[loss_str].values
        x = df_loss[pair[0]].values
        y = df_loss[pair[1]].values

        # Create grid values first.
        xi = np.linspace(x.min(), x.max(), ngridx)
        yi = np.linspace(y.min(), y.max(), ngridy)
        triang = tri.Triangulation(x, y)
        interpolator = tri.LinearTriInterpolator(triang, z)
        Xi, Yi = np.meshgrid(xi, yi)
        zi = interpolator(Xi, Yi)

        #from matplotlib import cm
        #from matplotlib.colors import ListedColormap, LinearSegmentedColormap
        #viridis = cm.get_cmap('viridis', 256)

        #
        #newcolors = viridis(np.linspace(0., 1., 256))
        #newcmp = ListedColormap(newcolors)

        cs.append(ax.pcolor(Xi,Yi,zi, vmin=vmin, vmax=vmax, norm=norm))

        #cs[-1].set_clim(vmin=vmin, vmax=vmax)
        ax.set_ylabel(pair[1])
        ax.set_xlabel(pair[0])

    return cs

In [ ]:
# Prepare the data frame
df_plt = _df.copy()
df_plt['latent_intermediate_dim'].replace([None], 0, inplace=True)
df_plt['beta_mutual'] = df_plt['beta_mutual'].apply(lambda x: np.log10(x))

In [ ]:
import matplotlib.colors as colors

f, ax = plt.subplots(1,3, figsize = (15, 5), dpi = 150)
# vmin, vmax = df_loss[loss_str].values.min(), df_loss[loss_str].values.max()
cs = plot_loss_surface('jsd_test_ab_vs_a', ['beta_mutual', 'latent_intermediate_dim', 'z_dim'],
                       df_plt, ax, vmin=0., vmax=1., norm=colors.PowerNorm(gamma=5.))
#f.colorbar(cs[-1], ax=ax[-1], shrink=0.9)
#f.set_clim(0., 1.)
f.colorbar(cs[-1], ax=ax)
print(cs[-1])
#f.colorbar()
plt.show()


In [ ]:
x, y, Z = df['beta_mutual'].values, df['latent_intermediate_dim'].values, df['h_loss'].values
x = x[np.newaxis, :]
y = y[:, np.newaxis]

In [ ]:
import plotly.express as px
fig = px.parallel_coordinates(df_plt, color="jsd_test_ab_vs_a",
                              dimensions=['latent_intermediate_dim', 'z_dim', 'beta_mutual'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=2)
fig.show()


In [ ]:

fig = px.parallel_coordinates(df_plt, color="z_dim",
                              dimensions=['h_loss', 'jsd_test_ab_vs_a', 'h_loss_mutual_0', 'h_loss_mutual_1'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=50)
fig.show()

In [ ]:
fig = px.parallel_coordinates(df_plt, color="latent_intermediate_dim",
                              dimensions=['h_loss', 'jsd_test_ab_vs_a', 'h_loss_mutual_0', 'h_loss_mutual_1'],
                              color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

In [ ]:
fig = px.parallel_coordinates(df_plt, color="beta_mutual",
                              dimensions=['h_loss', 'h_loss_mutual_0', 'h_loss_mutual_1', 'gnb_prec_ab_a', 'jsd_test_ab_vs_a'],
                              color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()


In [ ]:
fig = px.parallel_coordinates(df_plt, color="beta_mutual",
                              dimensions=['z_dim', 'latent_intermediate_dim', 'gnb_prec_ab_a', 'jsd_test_ab_vs_a'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              width=1920/2, height=1080/2)
fig.show()


In [ ]:

fig = px.parallel_coordinates(df_plt, color="beta_mutual",
                              dimensions=['h_loss', 'h_loss_mutual_0', 'h_loss_mutual_1', 'z_dim', 'latent_intermediate_dim', 'gnb_prec_ab_a', 'jsd_test_ab_vs_a'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              width=1920/1.5, height=1080/1.5)
fig.show()


In [ ]:
#h_loss_prior_0
#h_loss_reconstruction_0_0
#h_loss_reconstruction_1_0
#h_loss_reconstruction_2_0
#h_loss_reconstruction_2_1
#h_loss
fig = px.parallel_coordinates(df_plt, color="beta_mutual",
                              dimensions=['h_loss_reconstruction_0_0','h_loss_reconstruction_1_0','h_loss_reconstruction_2_0','h_loss_reconstruction_2_1','h_loss_prior_0','h_loss_prior_1','h_loss_prior_2', 'h_loss_mutual_0', 'h_loss_mutual_1', 'z_dim', 'latent_intermediate_dim', 'gnb_prec_a_ab', 'jsd_test_ab_vs_a'],
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              width=1920/1.5, height=1080/1.5)
fig.show()


# End of File